In [44]:
import numpy as np
import pandas as pd
import tensorflow as tf

train_data = pd.read_csv("titanic_train.csv")

UNUSED = ["Name", "Ticket", "Age", "Cabin", "Embarked", "Fare"]
train_data = train_data.drop(UNUSED, axis=1)

In [45]:
train_set = train_data.sample(frac=0.8, replace=False, random_state=42)
cv_set = train_data.loc[ set(train_data.index) - set(train_set.index)]

sex_feature = tf.feature_column.categorical_column_with_vocabulary_list(
    'Sex', ['female','male']
)

feature_columns = [ sex_feature ]

estimator = tf.estimator.LinearClassifier(
    feature_columns=feature_columns)

train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
      x=train_set.drop('Survived', axis=1),
      y=train_set.Survived,
      num_epochs=None,
      shuffle=True,
      target_column='target',
)

cv_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
      x=cv_set.drop('Survived', axis=1),
      y=cv_set.Survived,
      num_epochs=1,
      shuffle=False
)

estimator.train(input_fn=train_input_fn, steps=10)

scores = estimator.evaluate(input_fn=cv_input_fn)
print(f"Test Acc: {scores['accuracy']}")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/cp/pyymf_y13yq14gxtlh7_ngvc0000gn/T/tmpz7nx66u1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.


In [47]:
# Save model

sess = tf.compat.v1.Session

# builder = tf.compat.v1.saved_model.Builder("titanic")

test_data = pd.read_csv("titanic_test.csv")
test_data = test_data.drop(UNUSED, axis=1)
test_input_fn = tf.estimator.inputs.pandas_input_fn(
      x=test_data,
      num_epochs=1, # only to predict
      shuffle=False 
)

predictions = list(estimator.predict(input_fn=test_input_fn))
predicted_classes = [prediction['class_ids'][0] for prediction in predictions]
evaluation = test_data['PassengerId'].copy().to_frame()
evaluation["Survived"] = predicted_classes
evaluation.to_csv("evaluation_submission.csv", index=False)

#Tag the model, required for Go
builder.add_meta_graph_and_variables(sess, ["atag"])
builder.save()
sess.close()


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/cp/pyymf_y13yq14gxtlh7_ngvc0000gn/T/tmpz7nx66u1/model.ckpt-10
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InternalError'>, Unsupported object type float


InternalError: Unsupported object type float